## 1. PCC / 2. Mutual Info Scores / 3. LR / 4. RF
## 거리 적용은 머신러닝 파트 / 통계 분석에서 지역구별로 각각의 개수로 영향 평가

In [38]:
# Load your dataset
path = 'C:\\Users\\spa84\\'
data = pd.read_csv(path + '서울시_전세_건별통합본(23년, 건물연수 추가).csv', encoding='utf-8')
data = data[data['Building_Use'] == '아파트']

selected_columns = ['JS_Price', 'Building_Age']

# 컬럼 선택 및 랜덤 샘플링
data = data[selected_columns].sample(n=30000, random_state=42)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 200945 to 759996
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   JS_Price      30000 non-null  int64
 1   Building_Age  30000 non-null  int64
dtypes: int64(2)
memory usage: 703.1 KB


In [40]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

def normalize(data):
    for column in data.columns:
        if data[column].dtype == float:
            if column != 'JS_Price':
                data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())
    return data

data = normalize(data)

# Define X (features) and y (target variable)
X = data.drop(columns=['JS_Price'])
y = data['JS_Price']

# Step 1: Calculate Pearson Correlation Coefficients (PCC)
pcc_ranking = X.corrwith(y).abs().sort_values(ascending=False)

# Step 2: Compute Mutual Information Scores
mi_scores = mutual_info_regression(X, y)
mi_ranking = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Step 3: Fit a Linear Regression Model
linear_model = LinearRegression()
linear_model.fit(X, y)
linear_regression_coefficients = pd.Series(linear_model.coef_, index=X.columns).abs().sort_values(ascending=False)

# Step 4: Train a Random Forest Model
rf_model = RandomForestRegressor(n_estimators=1000, random_state=100)
rf_model.fit(X, y)
rf_feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

# Normalize each ranking
min_max_scaler = lambda x: (x - x.min()) / (x.max() - x.min())

normalized_pcc = min_max_scaler(pcc_ranking)
normalized_mi = min_max_scaler(mi_ranking)
normalized_linear = min_max_scaler(linear_regression_coefficients)
normalized_rf = min_max_scaler(rf_feature_importances)

# Define your weights for each ranking method (you can adjust these weights)
weight_pcc = 0.2
weight_mi = 0.2
weight_linear = 0.3
weight_rf = 0.3

# Combine the normalized rankings with weights
combined_ranking = (
    weight_pcc * normalized_pcc +
    weight_mi * normalized_mi +
    weight_linear * normalized_linear +
    weight_rf * normalized_rf
)

# Create a DataFrame with the combined ranking
combined_ranking_df = pd.DataFrame({
    'Feature': combined_ranking.index,
    'Combined_Ranking': combined_ranking.values
})

# Sort the features by the combined ranking
sorted_features = combined_ranking_df.sort_values(by='Combined_Ranking', ascending=False)

Ranking_APT=sorted_features

Ranking_APT

,Feature,Combined_Ranking
0,Building_Age,NaN
